<a href="https://colab.research.google.com/github/hanpec/test/blob/master/06%E7%AB%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ここに掲載されているコードは、  
『コピペで簡単実行！キテレツおもしろ自然言語処理～PythonとColaboratoryで身につく基礎の基礎』書籍の付録です。    
本ファイルの公開用URLや、本ファイル内のコードを、みだりに他の人へ共有しないでください。  

本ファイルは、書籍に記載のコードのコピペの手間を省くために作られました。  
本ファイルの公開用URLにアクセスすると、  
お手元のGoogleアカウントに本ファイルのコピーが作成されます。  
（つまり、コレをご覧になっているということは、コピーをご覧になっています）    
そのコピーは、あなただけのファイルとして生成されるため、    
書籍をお買い求め済みの方は、自由に変更したり実行したり保存したりして問題ありません。   

初期状態では、ご参考までに筆者が実行した出力結果を残してあります。  
ご自身で実行する際には、「編集」⇒「出力を全て消去」により、  
出力結果を消去してから実行なさると良いでしょう。  

※注：Googleの公式が作成したファイルではないため、実行時に  
「警告: このノートブックは Google が作成したものではありません。」  
という警告が出ます。  
気になる方は、ご自身の環境に都度コードをコピーして実行しても良いでしょう。




■  GoogleDriveのマウントコマンド：


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


■ 100MBの単純テキストデータのダウンロード：

In [ ]:
# ダウンロード用のフォルダを作成
!mkdir -p /content/drive/MyDrive/KITERETU
# 100MBの単純テキストデータのダウンロード
!curl -o /content/drive/MyDrive/KITERETU/text8ja42.txt https://storage.googleapis.com/nlp_youwht/text8/text8ja42.txt

# ランダムに生成しているため、何パターンか準備してあります。
# データを変更して試したい方は以下をご使用ください。
# !curl -o /content/drive/MyDrive/KITERETU/text8ja75.txt https://storage.googleapis.com/nlp_youwht/text8/text8ja75.txt
# !curl -o /content/drive/MyDrive/KITERETU/text8ja97.txt https://storage.googleapis.com/nlp_youwht/text8/text8ja97.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 95.3M  100 95.3M    0     0  53.4M      0  0:00:01  0:00:01 --:--:-- 53.4M


■ 冒頭部分を読んでみる：

In [ ]:
# -c バイト数（表示するバイト数を指定します。）
!head -c 80 "/content/drive/MyDrive/KITERETU/text8ja42.txt"

第8回ナショナル・ボード・オブ・レビュー賞は、1936年12月1

■ 全ての文字の間に半角スペースを入れる加工処理：

In [ ]:
%%time
import codecs
input_file_path = "/content/drive/MyDrive/KITERETU/text8ja42.txt"
output_file_path = "/content/drive/MyDrive/KITERETU/tmp_1kugiri.txt"

with codecs.open(input_file_path,"r", "utf-8") as f:
  for line in f.readlines():
    # 文字列を、１文字ごとのリストに変える（半角スペースは除く）
    chars = [c for c in line if c != u' ']
    with codecs.open(output_file_path,"a", "utf-8") as new_f:
      # リストの文字を半角スペースで繋げた文字列にする
      new_f.write(u' '.join(chars))



CPU times: user 5.69 s, sys: 2.55 s, total: 8.24 s
Wall time: 8.58 s


■ 作成したデータの冒頭部分を読んでみる：

In [ ]:
!head -c 80  "/content/drive/MyDrive/KITERETU/tmp_1kugiri.txt"

第 8 回 ナ シ ョ ナ ル ・ ボ ー ド ・ オ ブ ・ レ ビ ュ ー 

■ Char2Vecモデルの作成：

※2023年1月にColaboratoryのバージョンアップ/仕様変更が入り、  
　ログが出なくなり、また実行時間が、20分程度⇒70分程度、に   
　増える傾向にあるようです。   
　（使える実行環境はGoogleCloudの混雑状況次第ですので、状況により異なるようです）    
　Colaboratoryは90分操作が無いとセッションが切れてしまいますので、  
　稀にマウスを動かすなど様子を見ながらご実行ください。  


※2023年06月修正  
gensimのライブラリのバージョンがアップされ、  
それに伴いパラメータの名称が変更されたようで、  
size ⇒ vector_size  
iter ⇒ epochs  
に変更しました。  
また、実行時間が長すぎるため、３０分以内程度で出るよう少々パラメータを控えめにしました。

In [ ]:
%%time
import logging
from gensim.models import word2vec

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# １文字ごとに区切ったテキストデータを指定する
sentences = word2vec.Text8Corpus("/content/drive/MyDrive/KITERETU/tmp_1kugiri.txt")

# Word2Vecの学習実施
model = word2vec.Word2Vec(sentences,
                          vector_size = 40,
                          min_count = 5,
                          window = 20,
                          epochs = 5)

# できたモデルを保存する
model.save("/content/drive/MyDrive/KITERETU/c2v.model")


2021-08-15 14:57:10,134 : INFO : collecting all words and their counts
2021-08-15 14:57:10,142 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-08-15 14:57:18,711 : INFO : collected 7669 word types from a corpus of 35718977 raw words and 3572 sentences
2021-08-15 14:57:18,713 : INFO : Loading a fresh vocabulary
2021-08-15 14:57:18,732 : INFO : effective_min_count=5 retains 5230 unique words (68% of original 7669, drops 2439)
2021-08-15 14:57:18,734 : INFO : effective_min_count=5 leaves 35714493 word corpus (99% of original 35718977, drops 4484)
2021-08-15 14:57:18,756 : INFO : deleting the raw counts dictionary of 7669 items
2021-08-15 14:57:18,759 : INFO : sample=0.001 downsamples 74 most-common words
2021-08-15 14:57:18,762 : INFO : downsampling leaves estimated 25682535 word corpus (71.9% of prior 35714493)
2021-08-15 14:57:18,794 : INFO : estimated required memory for 5230 words and 60 dimensions: 5125400 bytes
2021-08-15 14:57:18,796 : INFO : resetti

CPU times: user 37min 47s, sys: 8.74 s, total: 37min 56s
Wall time: 19min 39s


■ できたモデルの内容を確認してみる：

In [ ]:
from gensim.models import word2vec

# 保存したモデルファイルの読み込み
model = word2vec.Word2Vec.load("/content/drive/MyDrive/KITERETU/c2v.model")

#出てきたモデルの確認
out = model.wv.most_similar(positive = [u'山'], topn=7)
print(out)
out = model.wv.most_similar(positive = [u'三'], topn=7)
print(out)
out = model.wv.most_similar(positive = [u'学'], topn=7)
print(out)
out = model.wv.most_similar(positive = [u'電'], topn=7)
print(out)
out = model.wv.most_similar(positive = [u'親'], topn=7)
print(out)


2021-08-15 15:16:47,804 : INFO : loading Word2Vec object from /content/drive/MyDrive/KITERETU/c2v.model
2021-08-15 15:16:47,844 : INFO : loading wv recursively from /content/drive/MyDrive/KITERETU/c2v.model.wv.* with mmap=None
2021-08-15 15:16:47,848 : INFO : setting ignored attribute vectors_norm to None
2021-08-15 15:16:47,850 : INFO : loading vocabulary recursively from /content/drive/MyDrive/KITERETU/c2v.model.vocabulary.* with mmap=None
2021-08-15 15:16:47,852 : INFO : loading trainables recursively from /content/drive/MyDrive/KITERETU/c2v.model.trainables.* with mmap=None
2021-08-15 15:16:47,855 : INFO : setting ignored attribute cum_table to None
2021-08-15 15:16:47,858 : INFO : loaded /content/drive/MyDrive/KITERETU/c2v.model
2021-08-15 15:16:47,877 : INFO : precomputing L2-norms of word weight vectors


[('麓', 0.714695930480957), ('梨', 0.6956164836883545), ('岳', 0.6803616285324097), ('津', 0.6678529381752014), ('翠', 0.6637073159217834), ('豊', 0.6632075905799866), ('峰', 0.6627737283706665)]
[('五', 0.8554501533508301), ('四', 0.8541182279586792), ('七', 0.8318808078765869), ('六', 0.8172636032104492), ('二', 0.7754413485527039), ('八', 0.7735314965248108), ('十', 0.705083966255188)]
[('塾', 0.5488768219947815), ('學', 0.5419449806213379), ('師', 0.5096393823623657), ('淨', 0.5020053386688232), ('諭', 0.49469608068466187), ('育', 0.4767284393310547), ('科', 0.462064266204834)]
[('磁', 0.6424942016601562), ('蒸', 0.630870521068573), ('炉', 0.6227524876594543), ('沸', 0.5132083296775818), ('筒', 0.5125842094421387), ('波', 0.5050251483917236), ('量', 0.4752518832683563)]
[('父', 0.8152867555618286), ('娘', 0.7752434015274048), ('母', 0.7695717811584473), ('妻', 0.7543565034866333), ('兄', 0.7506245970726013), ('戚', 0.7487218379974365), ('弟', 0.7450923323631287)]


■ 2つの漢字の融合結果を見る：

In [ ]:
from gensim.models import word2vec

def make_name(model, char1, char2):
  out = model.wv.most_similar(positive = [char1,char2], topn=7)
  print(char1 + "＋" + char2)
  print(out)

# 保存したモデルファイルの読み込み
model = word2vec.Word2Vec.load("/content/drive/MyDrive/KITERETU/c2v.model")

make_name(model, '雅','美')
make_name(model, '洋','千')
make_name(model, '隆','菜')
make_name(model, '治','恵')
make_name(model, '昌','杏')


2021-08-15 15:21:34,852 : INFO : loading Word2Vec object from /content/drive/MyDrive/KITERETU/c2v.model
2021-08-15 15:21:34,895 : INFO : loading wv recursively from /content/drive/MyDrive/KITERETU/c2v.model.wv.* with mmap=None
2021-08-15 15:21:34,897 : INFO : setting ignored attribute vectors_norm to None
2021-08-15 15:21:34,899 : INFO : loading vocabulary recursively from /content/drive/MyDrive/KITERETU/c2v.model.vocabulary.* with mmap=None
2021-08-15 15:21:34,901 : INFO : loading trainables recursively from /content/drive/MyDrive/KITERETU/c2v.model.trainables.* with mmap=None
2021-08-15 15:21:34,903 : INFO : setting ignored attribute cum_table to None
2021-08-15 15:21:34,905 : INFO : loaded /content/drive/MyDrive/KITERETU/c2v.model
2021-08-15 15:21:34,926 : INFO : precomputing L2-norms of word weight vectors


雅＋美
[('彦', 0.8101323246955872), ('也', 0.7648983597755432), ('澤', 0.7597938776016235), ('井', 0.7578713893890381), ('哉', 0.7495925426483154), ('櫻', 0.7396987080574036), ('吾', 0.7338852286338806)]
洋＋千
[('坡', 0.5517252087593079), ('里', 0.5489766597747803), ('浩', 0.5399837493896484), ('栗', 0.5322267413139343), ('斗', 0.5294121503829956), ('髙', 0.526018500328064), ('苗', 0.5235841274261475)]
隆＋菜
[('俊', 0.733126163482666), ('祐', 0.7119690179824829), ('淳', 0.708838701248169), ('彦', 0.6932514905929565), ('晃', 0.688141942024231), ('芳', 0.685152530670166), ('幸', 0.6842626929283142)]
治＋恵
[('孝', 0.6360815763473511), ('智', 0.6051125526428223), ('聡', 0.5923444628715515), ('昌', 0.5911316871643066), ('吉', 0.5842885971069336), ('田', 0.5829328298568726), ('寛', 0.5792694687843323)]
昌＋杏
[('淳', 0.8384228944778442), ('晃', 0.788501501083374), ('祐', 0.7810144424438477), ('吾', 0.7709164619445801), ('泰', 0.7599191069602966), ('孝', 0.7538734674453735), ('俊', 0.7528479099273682)]


■ 某国民的有名家族で計算：

In [ ]:
make_name(model, '栄','鱒')
make_name(model, '螺','鱒')
make_name(model, '波','船')
make_name(model, '平','船')


栄＋鱒
[('讃', 0.6155394315719604), ('苗', 0.5269569158554077), ('耕', 0.5074663758277893), ('榮', 0.49594932794570923), ('芥', 0.4821842312812805), ('肥', 0.4790078401565552), ('蔭', 0.47704628109931946)]
螺＋鱒
[('角', 0.5644404888153076), ('塊', 0.5524250864982605), ('鉤', 0.5351329445838928), ('尖', 0.532258152961731), ('鱗', 0.5312537550926208), ('吻', 0.5171781182289124), ('崗', 0.5153332948684692)]
波＋船
[('筏', 0.6712369918823242), ('雷', 0.6607320308685303), ('曳', 0.6317451000213623), ('艦', 0.6254606246948242), ('舶', 0.6198752522468567), ('艘', 0.6119551658630371), ('瑚', 0.592464029788971)]
平＋船
[('丸', 0.5333417654037476), ('樺', 0.5317610502243042), ('雷', 0.5300524830818176), ('瑚', 0.5281238555908203), ('礁', 0.5131152868270874), ('隻', 0.49410533905029297), ('空', 0.4917789101600647)]


■ 「みさえ」×「ひろし」⇒！？ ：


In [ ]:
make_name(model, '美','博')
make_name(model, '美','浩')
make_name(model, '美','弘')
make_name(model, '美','宏')
make_name(model, '美','寛')

make_name(model, '実','博')
make_name(model, '実','浩')
make_name(model, '実','弘')
make_name(model, '実','宏')
make_name(model, '実','寛')

make_name(model, '三','博')
make_name(model, '三','浩')
make_name(model, '三','弘')
make_name(model, '三','宏')
make_name(model, '三','寛')

美＋博
[('芸', 0.6467729806900024), ('館', 0.638398289680481), ('國', 0.564621090888977), ('富', 0.5583049058914185), ('壽', 0.5362088680267334), ('瑤', 0.5269686579704285), ('翁', 0.5158413648605347)]
美＋浩
[('彦', 0.8285688757896423), ('雅', 0.8135894536972046), ('也', 0.8128368854522705), ('宏', 0.7856806516647339), ('澤', 0.779481053352356), ('哉', 0.7773282527923584), ('晃', 0.7621461749076843)]
美＋弘
[('彦', 0.823050856590271), ('雅', 0.7418628931045532), ('俊', 0.7404600381851196), ('井', 0.7367808222770691), ('泰', 0.7285904884338379), ('隆', 0.724992036819458), ('吉', 0.7237218022346497)]
美＋宏
[('雅', 0.8265933394432068), ('彦', 0.8180125951766968), ('也', 0.7977275848388672), ('浩', 0.7764641046524048), ('井', 0.7578213214874268), ('哉', 0.7565929889678955), ('澤', 0.7558704614639282)]
美＋寛
[('徳', 0.7384751439094543), ('弘', 0.7298934459686279), ('斎', 0.7103860378265381), ('永', 0.6969512104988098), ('隆', 0.6924042701721191), ('仁', 0.6914777159690857), ('忠', 0.6853184700012207)]
実＋博
[('猷', 0.46067747473716736), ('

■ 「三」＋「浩」の上位１２位まで出力してみる：

In [ ]:
out = model.wv.most_similar(positive = [u'三', u'浩'], topn=12)
print(out)

[('巳', 0.8293680548667908), ('淳', 0.8262178301811218), ('吉', 0.8101320266723633), ('雅', 0.8087289929389954), ('田', 0.7970731258392334), ('郎', 0.7952207326889038), ('哉', 0.7890424728393555), ('晃', 0.7850452661514282), ('井', 0.7804944515228271), ('佑', 0.7724578380584717), ('嶋', 0.7715533375740051), ('俊', 0.7653734683990479)]
